### Предподготовка: подключение к скриптам в других папках

#### Активация прогрузки скриптов из папок

In [1]:
import os
import sys
import dotenv

# Загружаем переменные окружения из .env
dotenv.load_dotenv('paths.env')

# Добавляем PYTHONPATH в системный путь
for path in os.getenv("PYTHONPATH").split(":"):
    sys.path.append(os.path.abspath(path))

#### Импорт необходимых скриптов

In [2]:
# Прогрузка скрипта для возможности сохранять файлы в нужном формате + нужной директории
from data.load_data import *

In [ ]:
# Выгрузка необходимых библиотек
from utils.libraries_apply import *

# Прогрузка параметров для парсинга (список элементов, которые надо выгружать с сайта ЕРЗ)
from utils.parameters_erz import *

# Прогрузка класса парсинга сайта ЕРЗ
from utils.parsing_erz import *

#### Выгрузка и сохранение в формате .csv + ссылки для парсинга
На данный момент намного проще и быстрее залогиниться на сайте ЕРЗ вручную, а затем выгрузить список топ-ЖК с ссылками на них, файл при скачивании называем "top_zhk_@дата_на_которую_был_составлен_рейтинг"

In [4]:
path_notebooks = os.getcwd()
path_data_download = path_notebooks.replace('notebooks', 'data/raw')

In [5]:
# Поскольку пока что не настроен автоматический парсинг топ ЖК, 
# он выгружается вручную, а после выгружается сама таблица для последующей обработки
name_file = path_data_download + '/top_zhk_31.10.2024.xlsx'

In [6]:
from openpyxl import load_workbook
import csv

# Выгрузка файла
wb = load_workbook(name_file)
ws = wb.active

# Путь сохранения файла в формате .csv
csv_file_path = path_data_download + '/top_zhk_31.10.2024.csv'

# Открываем CSV файл для записи с UTF-8
with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Пробегаемся по всем строкам и обрабатываем ячейки
    for row in ws.iter_rows():
        row_data = []
        for cell in row:
            # Проверяем на наличие гиперссылки
            try:
                if cell.hyperlink and cell.hyperlink.target:
                    # Если есть гиперссылка, добавляем её к значению ячейки
                    cell_value = f'{cell.value or ''} | {cell.hyperlink.target}'
                else:
                    cell_value = cell.value
            except:
                # Обработка ошибок на случай пустых значений
                cell_value = cell.value  
            
            # Добавляем значение в текущую строку
            row_data.append(cell_value)
        
        # Записываем строку в CSV файл
        csv_writer.writerow(row_data)
        
print(f'Файл успешно сохранён как CSV: {csv_file_path}')


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Файл успешно сохранён как CSV: /Users/leonbah/Desktop/Python/ВКРС/Main Repo/data/raw/top_zhk_31.10.2024.csv


In [7]:
# Проверяем последством выгрузки
top_zhk = pd.read_csv(csv_file_path)

hyper = (top_zhk['Наименование ЖК'].str.rsplit(" | ", expand=True))
hyper.columns = ['Наименование ЖК', 'Ссылки']

top_zhk['Наименование ЖК'] = hyper['Наименование ЖК']
top_zhk['Ссылки'] = hyper['Ссылки']

### Запуск парсинга

#### Парсинг застройщиков  
Код на доработке, поскольку допускаются ошибки при выгрузке, которые не в явном виде и которые сложно отследить

In [8]:
# ---

#### Парсинг ЖК

In [9]:
# Создание словаря {'Наименование ЖК': 'ссылка на ЕРЗ сайт'}
zhk_links = {}
for i in range(len(top_zhk)):
    zhk_links[top_zhk['Наименование ЖК'][i]] = top_zhk['Ссылки'][i]
    
# Отдельные параметры, на которые стоит обращать внимание
range_parameters = range(6, 24)
# parameters_box = "//div[@class='flex m-t-10 ng-tns-c62-{} ng-trigger ng-trigger-fade ng-star-inserted']"
labels_box = "//div[@class='left-column flex flex-middle']"
values_box = "//div[@class='f-b-35 descr']"

In [ ]:
def parsing_page(link: str, name_estate: str) -> dict:
    
    # Инициируем настройки для хрома
    chrome_options = Options()
     
    chrome_options.add_argument("--headless=new")  # <-- Фоновый режим
    chrome_options.add_argument("--disable-gpu")   # <-- Отключение GPU (для headless)
    chrome_options.add_argument("--no-sandbox")    # <-- Отключение sandbox (для некоторых систем)
    
    # Устанавливаем/обновляем хром-драйвер, если надо + встраиваем настройки браузера
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service = service, options = chrome_options)
    
    # Отправляем запрос на запуск функции
    driver.get(link)
    wait = WebDriverWait(driver, 10)
    
    # Запуск кода для визуальной проверки структуры сайта
    # html = driver.page_source
    # soup = BeautifulSoup(html, 'html.parser')
    # print(BeautifulSoup.prettify(soup))
    
    try:
        
        # Поиск наименований потенциальных параметров ЖК
        labels_list = wait.until(EC.presence_of_all_elements_located((By.XPATH, labels_box)))
        labels_text_list = [el.text for el in labels_list]
        
        # Поиск значений потенциальных параметров ЖК
        values_list = wait.until(EC.presence_of_all_elements_located((By.XPATH, values_box)))
        values_text_list = [el.text for el in values_list]
        
        # Создание словаря параметров ЖК
        parameters = {}
        for i in range(len(labels_text_list)):
            parameters[labels_text_list[i]] = values_text_list[i]

        print(name_estate)
        
    except:
        parameters = {}
        print(name_estate, '<-- на уровне браузера')
        
    return parameters

In [ ]:
# Выгрузка класса парсинга для ЖК
parsing_class = Parsing

# Запуск цикла по парсингу каждой ссылки ЖК
for zhk_name in zhk_links:
    sp = parsing_class._process_retry(parsing_class._link_get, zhk_links[zhk_name])
    
    